In [12]:
# Import 
import os
import time
import json
import datetime
import zipfile
import numpy as np
import pandas as pd

from IPython.display import display

from pathlib import Path
from tyssue import config
from tyssue import Sheet
from tyssue.geometry.sheet_geometry import ClosedSheetGeometry as geom
from tyssue.io import hdf5
from tyssue.io.hdf5 import load_datasets
from tyssue.dynamics import SheetModel as basemodel
from tyssue.dynamics.apoptosis_model import SheetApoptosisModel as model



In [2]:
def open_sheet(dirname, t, data_names=['vert', 'edge', 'face', 'cell']):
    """Open hdf5 file

    Open HDF5 file correspond to t time from dirname directory.

    Parameters
    ----------
    directory : str
        complete directory path
    t : int
        time step
    """
    file_name = 'invagination_{:04d}.hf5'.format(t)
    dsets = hdf5.load_datasets(os.path.join(dirname, file_name),
                               data_names=data_names)

    specs = config.geometry.cylindrical_sheet()
    sheet = Sheet('ellipse', dsets, specs)
    return sheet


In [13]:
SIM_DIR = Path('/home/admin-suz/Documents/Simulations/20190827-QSSolver-Prepolarized')
dirname = SIM_DIR/'1.08_constriction_0_radialtension0/'

In [45]:

for t in range(0, 200):
    try:
        sheet = open_sheet(dirname,t)
        sheet.face_df['time']=t
        sheet.face_df.to_csv(os.path.join(dirname, "face.csv"), mode='a')
        sheet.edge_df.to_csv(os.path.join(dirname, "edge.csv"), mode='a')
        sheet.vert_df.to_csv(os.path.join(dirname, "vert.csv"), mode='a')
    except Exception:
        pass

pd.Series(sheet.settings).to_csv(os.path.join(dirname, 'settings.csv'))

#Faire le zip avec la bibliothèque shutil plutôt ? 
with zipfile.ZipFile(os.path.join(dirname, str(dirname).split('/')[-1]+'.zip'), 'w') as zip :
    zip.write(os.path.join(dirname, "face.csv"), arcname='face.csv')
    zip.write(os.path.join(dirname, "edge.csv"), arcname='edge.csv')
    zip.write(os.path.join(dirname, "vert.csv"), arcname='vert.csv')
    zip.write(os.path.join(dirname, "settings.csv"), arcname='settings.csv')
    
os.remove(os.path.join(dirname, "settings.csv"))
os.remove(os.path.join(dirname, "face.csv"))
os.remove(os.path.join(dirname, "edge.csv"))
os.remove(os.path.join(dirname, "vert.csv"))
